## **Omdena LATAM Fake News Detector Model**

In [1]:
import pandas as pd
import numpy as np
import re
import string
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV



In [2]:
df_train = pd.read_parquet('/content/16eac85914e04a167f656392c2850aac197133366a5d953a9499702e5360d7a4')
df_test = pd.read_parquet('/content/df1dafd7a13176813587af8ca6367f723827be4a93522907e443925340f4e445')

In [3]:
df_train.head()

,Annotation_action,Article_ID,Author,Content,Corrected_label,Date_posted,Link,Prediction,Prediction_prob,Preprocessed_text,Rescaled_prediction_prob,Source,Text,Title,Type,__index_level_0__
0,Don't Annotate,2858,None,En un artículo difundido en su sitio personal ...,Fake,2022-03-01,None,Fake,0.632735,artículo difundido sitio personal coordinador...,0.132735,proceso,Source:proceso\n\n Prediction:Fake\n\n Content...,,Tweet,140
1,Don't Annotate,2005,None,"🔴 El líder nacional de Morena, @mario_delgado ...",True,2022-01-19,None,True,0.441359,líder nacional Morena @mario_delgado llamar I...,0.058641,Excelsior,Source:Excelsior\n\n Prediction:True\n\n Conte...,,Tweet,164
2,Annotate,453,None,NUEVA YORK.- Ben Simmons aseguró el martes qu...,Fake,None,https://vanguardia.com.mx/deportes/ben-simmons...,Fake,0.564037,YORK.- Ben Simmons martes problema mental inic...,0.064037,La Vanguardia,Source:La Vanguardia\n\n Prediction:Fake\n\n C...,Ben Simmons da ‘su verdad’ sobre su salida de ...,Web Article,1449
3,Don't Annotate,590,None,Un estudio realizado por la empresa tecnológi...,True,None,http://www.elnuevosiglo.com.co/articulos/02-14...,Fake,0.584732,estudio empresa tecnológico gamificación Nawai...,0.084732,El Nuevo Siglo,Source:El Nuevo Siglo\n\n Prediction:Fake\n\n ...,Destacan habilidades profesionales de perfiles...,Web Article,29
4,Don't Annotate,1008,None,\nLa Subcomisión Política leyó ayer públic...,True,2020-07-28,https://diario.elmundo.sv/Política/subcomision...,True,0.357733,Subcomisión Política leer públicamente informe...,0.142267,El Mundo,Source:El Mundo\n\n Prediction:True\n\n Conten...,Subcomisión avaló las 75 candidaturas a la C...,Web Article,65


In [4]:
df_test.head()

,Annotation_action,Article_ID,Author,Content,Corrected_label,Date_posted,Link,Prediction,Prediction_prob,Preprocessed_text,Rescaled_prediction_prob,Source,Text,Title,Type,__index_level_0__
0,Don't Annotate,636,None,"\n\n\r\nMalcolm Cartagena, capacitador del Tri...",True,2017-12-30,https://diario.elmundo.sv/Política/resuelve-tu...,True,0.410082,Malcolm Cartagena capacitador Tribunal Supremo...,0.089918,El Mundo,Source:El Mundo\n\n Prediction:True\n\n Conten...,Resuelve tus dudas sobre las JRV en 12 preguntas,Web Article,1593
1,Don't Annotate,536,None,\n\n\r\nLa Comisión Especial Electoral del FM...,True,2019-06-21,https://diario.elmundo.sv/Política/cce-declara...,Fake,0.575066,Comisión Especial Electoral FMLN declarar maña...,0.075066,El Mundo,Source:El Mundo\n\n Prediction:Fake\n\n Conten...,CEE declara a Ortiz como nuevo secretario gene...,Web Article,1486
2,Don't Annotate,2184,None,Tras el el enfrentamiento entre simpatizantes ...,True,2018-04-01,None,Fake,0.564037,enfrentamiento simpatizante PRD Morena previo ...,0.064037,SinEmbargoMX,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...,,Tweet,532
3,Don't Annotate,2914,None,La @Coparmex advirtió que a la @CFEmx le urge ...,Fake,2021-12-10,None,Fake,0.649744,@coparmex advertir @cfemx urge reestructura fi...,0.149744,proceso,Source:proceso\n\n Prediction:Fake\n\n Content...,,Tweet,1153
4,Annotate,261,None,"\nMichelle Sol, alcaldesa de Nuevo Cuscatlán....",True,2019-05-10,https://diario.elmundo.sv/Política/michelle-so...,True,0.462732,Michelle Sol alcaldesa Cuscatlán alcaldesa Cus...,0.037268,El Mundo,Source:El Mundo\n\n Prediction:True\n\n Conten...,Michelle Sol depondrá cargo en alcaldía tras...,Web Article,132


In [5]:
#df_news = pd.read_csv("/content/fake-news_preprocessed-corrected_train.csv")

In [6]:
#df_news.head()

In [7]:
df_train = df_train[["Title", "Content", "Prediction", "Text"]]
df_test = df_test[["Title", "Content", "Prediction", "Text"]]

In [8]:
df_train.head()

,Title,Content,Prediction,Text
0,,En un artículo difundido en su sitio personal ...,Fake,Source:proceso\n\n Prediction:Fake\n\n Content...
1,,"🔴 El líder nacional de Morena, @mario_delgado ...",True,Source:Excelsior\n\n Prediction:True\n\n Conte...
2,Ben Simmons da ‘su verdad’ sobre su salida de ...,NUEVA YORK.- Ben Simmons aseguró el martes qu...,Fake,Source:La Vanguardia\n\n Prediction:Fake\n\n C...
3,Destacan habilidades profesionales de perfiles...,Un estudio realizado por la empresa tecnológi...,Fake,Source:El Nuevo Siglo\n\n Prediction:Fake\n\n ...
4,Subcomisión avaló las 75 candidaturas a la C...,\nLa Subcomisión Política leyó ayer públic...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...


In [9]:
df_test.head()

,Title,Content,Prediction,Text
0,Resuelve tus dudas sobre las JRV en 12 preguntas,"\n\n\r\nMalcolm Cartagena, capacitador del Tri...",True,Source:El Mundo\n\n Prediction:True\n\n Conten...
1,CEE declara a Ortiz como nuevo secretario gene...,\n\n\r\nLa Comisión Especial Electoral del FM...,Fake,Source:El Mundo\n\n Prediction:Fake\n\n Conten...
2,,Tras el el enfrentamiento entre simpatizantes ...,Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...
3,,La @Coparmex advirtió que a la @CFEmx le urge ...,Fake,Source:proceso\n\n Prediction:Fake\n\n Content...
4,Michelle Sol depondrá cargo en alcaldía tras...,"\nMichelle Sol, alcaldesa de Nuevo Cuscatlán....",True,Source:El Mundo\n\n Prediction:True\n\n Conten...


In [10]:
fake_df = df_test[df_test["Prediction"] == "Fake"]
true_df = df_test[df_test["Prediction"] == "True"]

In [11]:
fake_df.head()

,Title,Content,Prediction,Text
1,CEE declara a Ortiz como nuevo secretario gene...,\n\n\r\nLa Comisión Especial Electoral del FM...,Fake,Source:El Mundo\n\n Prediction:Fake\n\n Conten...
2,,Tras el el enfrentamiento entre simpatizantes ...,Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...
3,,La @Coparmex advirtió que a la @CFEmx le urge ...,Fake,Source:proceso\n\n Prediction:Fake\n\n Content...
5,,.@lopezobrador_ agregó que sus opositores en M...,Fake,Source:elsolde_mexico\n\n Prediction:Fake\n\n ...
6,Alberto quiere inventar un ministerio para Pé...,Alberto Fernández quiere crearle un ministeri...,Fake,Source:La Política\n\n Prediction:Fake\n\n Con...


In [12]:
true_df.head()

,Title,Content,Prediction,Text
0,Resuelve tus dudas sobre las JRV en 12 preguntas,"\n\n\r\nMalcolm Cartagena, capacitador del Tri...",True,Source:El Mundo\n\n Prediction:True\n\n Conten...
4,Michelle Sol depondrá cargo en alcaldía tras...,"\nMichelle Sol, alcaldesa de Nuevo Cuscatlán....",True,Source:El Mundo\n\n Prediction:True\n\n Conten...
7,CSJ ordena juicio a expresidente del BMI y exv...,\n\n\r\nLa Corte Suprema de Justicia (CSJ) ord...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...
9,HRW acusa a Bukele de socavar las bases democr...,\n Human Rights Watch (HRW) señala al ...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...
12,Grieta en carretera a Monclova ya existía; Re...,Se realizó un estudio a la estructura del pue...,True,Source:La Vanguardia\n\n Prediction:True\n\n C...


In [13]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 1 to 309
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       152 non-null    object
 1   Content     152 non-null    object
 2   Prediction  152 non-null    object
 3   Text        152 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [14]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 308
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       158 non-null    object
 1   Content     158 non-null    object
 2   Prediction  158 non-null    object
 3   Text        158 non-null    object
dtypes: object(4)
memory usage: 6.2+ KB


In [15]:
fake_df['class'] = 1
true_df['class'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
testing_csv = pd.concat([fake_df[:10], true_df[:10]], axis=0)
testing_csv.to_csv('testing.csv')

In [17]:
df_merged = pd.concat([fake_df, true_df], axis=0)

In [18]:
df_merged.sample(10).head(10)

,Title,Content,Prediction,Text,class
132,Virginia Eubanks: “La tecnología nos ofrece u...,Supera en 17.000 personas la población de Tro...,True,Source:El País\n\n Prediction:True\n\n Content...,0
279,,"El escenario sanitario, político, económico y ...",Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...,1
259,Alberto respalda a Vilma en la pelea con los m...,Alberto Fernández respaldó a Vilma Ibarra en...,Fake,Source:La Política\n\n Prediction:Fake\n\n Con...,1
113,,"Adán Augusto López, titular de la Segob, infor...",True,Source:SinEmbargoMX\n\n Prediction:True\n\n Co...,0
183,,➡️ El TEPJF ordenó al INE contestarle a Morena...,True,Source:Excelsior\n\n Prediction:True\n\n Conte...,0
53,,"#Elecciones2021MX del #6DeJunio, triunfo ""pers...",Fake,Source:LaJornada\n\n Prediction:Fake\n\n Conte...,1
23,,¿Una relación en pausa? | AMLO vs. España: La ...,Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...,1
199,,"PRI, PAN y PRD piden claridad en costo de inde...",True,Source:proceso\n\n Prediction:True\n\n Content...,0
308,"Recomiendan subir el bono de policías, soldad...",\n\n\r\nLa Asamblea Legislativa aprobó con 79...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...,0
275,,El Canciller Ebrard ya ocupaba un lugar prefer...,True,Source:SinEmbargoMX\n\n Prediction:True\n\n Co...,0


In [19]:
df_merged_new = df_merged[["Content", "class"]]
df_merged_new = df_merged.sample(frac=1)


In [20]:
df_merged_new.head()

,Title,Content,Prediction,Text,class
46,,#MartínCaparrós presentó su libró #Ñamérica en...,Fake,Source:LaJornada\n\n Prediction:Fake\n\n Conte...,1
2,,Tras el el enfrentamiento entre simpatizantes ...,Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...,1
158,Manganitas ∙ AFA,“... AMLO les pagará a los delincuentes para ...,Fake,Source:La Vanguardia\n\n Prediction:Fake\n\n C...,1
148,Abogados: las sanciones a las aerolíneas debe...,\nEl Gobierno sigue exigiendo la prueba covid1...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...,0
90,\r\n\r\nSe instalarán 602 casillas para la re...,<p>El titular de la<b> Junta Local Ejecutiva d...,True,Source:Milenio\n\n Prediction:True\n\n Content...,0


In [21]:
df_merged_new.isnull().sum()

Title         0
Content       0
Prediction    0
Text          0
class         0
dtype: int64

In [22]:
df_merged_new.dropna()

,Title,Content,Prediction,Text,class
46,,#MartínCaparrós presentó su libró #Ñamérica en...,Fake,Source:LaJornada\n\n Prediction:Fake\n\n Conte...,1
2,,Tras el el enfrentamiento entre simpatizantes ...,Fake,Source:SinEmbargoMX\n\n Prediction:Fake\n\n Co...,1
158,Manganitas ∙ AFA,“... AMLO les pagará a los delincuentes para ...,Fake,Source:La Vanguardia\n\n Prediction:Fake\n\n C...,1
148,Abogados: las sanciones a las aerolíneas debe...,\nEl Gobierno sigue exigiendo la prueba covid1...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...,0
90,\r\n\r\nSe instalarán 602 casillas para la re...,<p>El titular de la<b> Junta Local Ejecutiva d...,True,Source:Milenio\n\n Prediction:True\n\n Content...,0
...,...,...,...,...,...
146,Israel califica de abuso cruel terapias de con...,El gobierno de Israel prohibió a trabajadores...,True,Source:La Vanguardia\n\n Prediction:True\n\n C...,0
291,,El TEEM quita a Morena 10 diputados; reparte l...,Fake,Source:proceso\n\n Prediction:Fake\n\n Content...,1
30,Diputados aprueban seguro médico de Asamblea ...,\nEl seguro médico fue aprobado con votos de ...,True,Source:El Mundo\n\n Prediction:True\n\n Conten...,0
257,¡Abríguese!... Frente Frío 30 golpeará al t...,Así mismo se pronostica un nuevo descenso de ...,True,Source:La Vanguardia\n\n Prediction:True\n\n C...,0


In [23]:
df_merged_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 46 to 216
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       310 non-null    object
 1   Content     310 non-null    object
 2   Prediction  310 non-null    object
 3   Text        310 non-null    object
 4   class       310 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 14.5+ KB


In [24]:
df_merged_new = df_merged_new[['Content', 'class']]

In [25]:
df_merged_new.head()

,Content,class
46,#MartínCaparrós presentó su libró #Ñamérica en...,1
2,Tras el el enfrentamiento entre simpatizantes ...,1
158,“... AMLO les pagará a los delincuentes para ...,1
148,\nEl Gobierno sigue exigiendo la prueba covid1...,0
90,<p>El titular de la<b> Junta Local Ejecutiva d...,0


In [26]:
df_merged_new

,Content,class
46,#MartínCaparrós presentó su libró #Ñamérica en...,1
2,Tras el el enfrentamiento entre simpatizantes ...,1
158,“... AMLO les pagará a los delincuentes para ...,1
148,\nEl Gobierno sigue exigiendo la prueba covid1...,0
90,<p>El titular de la<b> Junta Local Ejecutiva d...,0
...,...,...
146,El gobierno de Israel prohibió a trabajadores...,0
291,El TEEM quita a Morena 10 diputados; reparte l...,1
30,\nEl seguro médico fue aprobado con votos de ...,0
257,Así mismo se pronostica un nuevo descenso de ...,0


In [27]:
x = df_merged_new['Content'].fillna('')
y = df_merged_new['class']



In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.30)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorization = TfidfVectorizer()
vector_train = vectorization.fit_transform(X_train)
vector_test = vectorization.transform(X_test)


### **Logistic Regression**


In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
LR = LogisticRegression()
LR.fit(vector_train, y_train)


LogisticRegression()

In [33]:
LR.score(vector_test, y_test)

0.7311827956989247

In [34]:
pred_LR = LR.predict(vector_test)

In [35]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.78      0.77      0.77        56
           1       0.66      0.68      0.67        37

    accuracy                           0.73        93
   macro avg       0.72      0.72      0.72        93
weighted avg       0.73      0.73      0.73        93



## **Decision Tree**


In [36]:
from sklearn.tree import DecisionTreeClassifier

In [37]:
DT = DecisionTreeClassifier()
DT.fit(vector_train, y_train)


DecisionTreeClassifier()

In [38]:
DT.score(vector_test, y_test)

0.7634408602150538

In [39]:
pred_DT = DT.predict(vector_test)

In [40]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       0.85      0.73      0.79        56
           1       0.67      0.81      0.73        37

    accuracy                           0.76        93
   macro avg       0.76      0.77      0.76        93
weighted avg       0.78      0.76      0.77        93



### **Gradient Boost**

In [41]:
from sklearn.ensemble import GradientBoostingClassifier

In [42]:
GBR = GradientBoostingClassifier()
GBR.fit(vector_train, y_train)

GradientBoostingClassifier()

In [43]:
GBR.score(vector_test, y_test)

0.7741935483870968

In [44]:
pred_GBR = GBR.predict(vector_test)

In [45]:
print(classification_report(y_test, pred_GBR))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80        56
           1       0.68      0.81      0.74        37

    accuracy                           0.77        93
   macro avg       0.77      0.78      0.77        93
weighted avg       0.79      0.77      0.78        93



### **Random Forest**

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
RFC = RandomForestClassifier()
RFC.fit(vector_train, y_train)

RandomForestClassifier()

In [48]:
RFC.score(vector_test, y_test)

0.6881720430107527

In [49]:
pred_RFC = RFC.predict(vector_test)

In [50]:
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73        56
           1       0.60      0.68      0.63        37

    accuracy                           0.69        93
   macro avg       0.68      0.69      0.68        93
weighted avg       0.70      0.69      0.69        93



## **Ridge Regression w/GridSearch for Hyperparameter Optimization**

In [51]:
model = Ridge()

In [52]:
cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=123)

In [53]:
Ridge(alpha=1.0, fit_intercept=False, normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='sparse_cg', random_state=None,)

Ridge(fit_intercept=False, normalize=False, solver='sparse_cg')

In [54]:
param = {
    'solver':['svd', 'cholesky', 'lsqr', 'sag'],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
    'fit_intercept':[True, False],
    'normalize':[True, False]
}

In [55]:
# define search
search = GridSearchCV(model, param, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv)
# execute search
result = search.fit(vector_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
3200 fits failed out of a total of 6400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
800 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py", line 1011, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py", line 724, in fit
    "or set `fit_intercept=False`".format(self.solver)
Value

In [56]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: -0.3853081100758665
Best Hyperparameters: {'alpha': 1e-05, 'fit_intercept': True, 'normalize': False, 'solver': 'sag'}


In [57]:
!pip install https://github.com/ydataai/pandas-profiling/archive/master.zip --quiet
!pip install scipy --quiet

In [58]:
from pandas_profiling import ProfileReport

In [59]:
#restart runtime and rerun the notebook
profile = ProfileReport(df_merged_new, title='Profile Report', html={'style':{'full_width':True}})

In [60]:
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [61]:
regressors = [
    KNeighborsRegressor(),
    GradientBoostingRegressor(),
    KNeighborsRegressor(),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
    LinearRegression(),
    Lasso(),
    Ridge()
]

In [62]:
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

In [65]:
head = 10
for model in regressors[:head]:
    start = time()
    model.fit(vector_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(vector_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))


KNeighborsRegressor()
	Training time: 0.004s
	Prediction time: 0.019s
	Explained variance: 0.022857142857142576
	Mean absolute error: 0.41935483870967744
	R2 score: 0.014343629343629094
GradientBoostingRegressor()
	Training time: 2.314s
	Prediction time: 0.001s
	Explained variance: 0.3635802796047807
	Mean absolute error: 0.3076113735678133
	R2 score: 0.3422375607090892
KNeighborsRegressor()
	Training time: 0.001s
	Prediction time: 0.005s
	Explained variance: 0.022857142857142576
	Mean absolute error: 0.41935483870967744
	R2 score: 0.014343629343629094
ExtraTreesRegressor()
	Training time: 4.126s
	Prediction time: 0.033s
	Explained variance: 0.31604102316602334
	Mean absolute error: 0.33473118279569886
	R2 score: 0.25121085907335916
RandomForestRegressor()
	Training time: 3.949s
	Prediction time: 0.053s
	Explained variance: 0.3675744208494208
	Mean absolute error: 0.3410752688172043
	R2 score: 0.31814662162162144
DecisionTreeRegressor()
	Training time: 0.032s
	Prediction time: 0.001s
	

In [66]:
parameters = { 'loss' : ['ls', 'lad', 'huber', 'quantile'],
              'learning_rate' : (0.05,0.25,0.50,1),
              'criterion' : ['friedman_mse', 'mse', 'mae'],
              'max_features' : ['auto', 'sqrt', 'log2']
             }

In [67]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [68]:
grid = GridSearchCV(GradientBoostingRegressor(),parameters)
model = grid.fit(vector_train, y_train)
print(model.best_params_,'\n')
print(model.best_estimator_,'\n')

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:370: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist

{'criterion': 'mse', 'learning_rate': 0.25, 'loss': 'huber', 'max_features': 'auto'} 

GradientBoostingRegressor(criterion='mse', learning_rate=0.25, loss='huber',
                          max_features='auto') 



/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py:363: FutureWarning: 

In [69]:
!pip install pickle4 --quiet


In [70]:
import pickle

with open('GBR.pickle','wb') as modelFile:
     pickle.dump(pred_GBR, modelFile)

In [71]:
import sys
print(sys.modules.keys())

dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'warnings', 're', 'enum', 'types', '_collections', 'sre_compile', '_sre', 'sre_parse', 'sre_constants', 'functools', '_functools', 'collections', '_collections_abc', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', 'copyreg', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_sitebuiltins', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'google', 'google.cloud', 'google.logging', 'mpl_toolkits', 'sphinxcontrib', 'sitecustomize', 'runpy', 'pkgutil', 'weakref', '_weakrefset', 'ipykernel', 'ipykernel._version', 'ipykernel

In [72]:
!pip install session-info

     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8048 sha256=01b3aeb970703a644f27d80ab6a326037105724fb6071993db37e60912f4fbad
  Stored in directory: /root/.cache/pip/wheels/bd/ad/14/6a42359351a18337a8683854cfbba99dd782271f2d1767f87f
Successfully built session-info


In [73]:
import session_info
session_info.show(html=False)

-----
numpy               1.21.5
pandas              1.3.5
pandas_profiling    3.1.1
scipy               1.7.3
session_info        1.0.0
sklearn             1.0.2
-----
IPython             5.5.0
jupyter_client      5.3.5
jupyter_core        4.9.2
notebook            5.3.1
-----
Python 3.7.13 (default, Mar 16 2022, 17:37:17) [GCC 7.5.0]
Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
-----
Session information updated at 2022-04-17 09:03
